In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


## 1. Load Train Data and Test Data

In [2]:
data_train = pd.read_csv("../input/spaceship-titanic/train.csv")
data_test = pd.read_csv("../input/spaceship-titanic/test.csv")

In [3]:
spaceship_titanic = data_train.append(data_test)
spaceship_titanic

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,NaN
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,NaN
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,NaN
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,NaN


In [4]:
spaceship_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12970 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   12970 non-null  object 
 1   HomePlanet    12682 non-null  object 
 2   CryoSleep     12660 non-null  object 
 3   Cabin         12671 non-null  object 
 4   Destination   12696 non-null  object 
 5   Age           12700 non-null  float64
 6   VIP           12674 non-null  object 
 7   RoomService   12707 non-null  float64
 8   FoodCourt     12681 non-null  float64
 9   ShoppingMall  12664 non-null  float64
 10  Spa           12686 non-null  float64
 11  VRDeck        12702 non-null  float64
 12  Name          12676 non-null  object 
 13  Transported   8693 non-null   object 
dtypes: float64(6), object(8)
memory usage: 1.5+ MB


In [5]:
spaceship_titanic.index = spaceship_titanic['PassengerId']
spaceship_titanic = spaceship_titanic.drop(['Name'], axis=1)

## 2. Fix Categorical Data

In [6]:
spaceship_titanic[['GroupId', 'PassengerIdOrder']] = spaceship_titanic['PassengerId'].str.split('_', expand=True)
spaceship_titanic['IsAlone'] = ~spaceship_titanic['GroupId'].duplicated(keep=False)
spaceship_titanic['IsAlone'] = spaceship_titanic['IsAlone'].map({True:1, False:0})
spaceship_titanic = spaceship_titanic.drop(['PassengerId', 'GroupId', 'PassengerIdOrder'], axis=1)

In [7]:
spaceship_titanic[['Deck', 'Num', 'Side']] = spaceship_titanic['Cabin'].str.split('/', expand=True)
spaceship_titanic = spaceship_titanic.drop('Cabin', axis=1)

In [8]:
deck = pd.get_dummies(spaceship_titanic["Deck"])
deck.columns = ['Deck' + col for col in deck.columns]
spaceship_titanic = spaceship_titanic.drop('Deck',axis = 1)
spaceship_titanic = spaceship_titanic.join(deck)

In [9]:
num = pd.get_dummies(spaceship_titanic["Num"])
num.columns = ['Num' + col for col in num.columns]
spaceship_titanic = spaceship_titanic.drop('Num',axis = 1)
spaceship_titanic = spaceship_titanic.join(num)

In [10]:
side = pd.get_dummies(spaceship_titanic["Side"])
side.columns = ['Side' + col for col in side.columns]
spaceship_titanic = spaceship_titanic.drop('Side',axis = 1)
spaceship_titanic = spaceship_titanic.join(side)

In [11]:
home_planet = pd.get_dummies(spaceship_titanic["HomePlanet"])
spaceship_titanic = spaceship_titanic.drop('HomePlanet',axis = 1)
spaceship_titanic = spaceship_titanic.join(home_planet)

In [12]:
cryo_sleep = pd.get_dummies(spaceship_titanic["CryoSleep"])
cryo_sleep.columns = ['CryoSleepTrue', 'CryoSleepFalse']
spaceship_titanic = spaceship_titanic.drop('CryoSleep',axis = 1)
spaceship_titanic = spaceship_titanic.join(cryo_sleep)

In [13]:
destination = pd.get_dummies(spaceship_titanic["Destination"])
spaceship_titanic = spaceship_titanic.drop('Destination',axis = 1)
spaceship_titanic = spaceship_titanic.join(destination)

In [14]:
vip = pd.get_dummies(spaceship_titanic['VIP'])
vip.columns = ['VIPTrue', 'VIPFalse']
spaceship_titanic = spaceship_titanic.drop('VIP',axis = 1)
spaceship_titanic = spaceship_titanic.join(vip)

In [15]:
spaceship_titanic.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,IsAlone,DeckA,DeckB,...,Earth,Europa,Mars,CryoSleepTrue,CryoSleepFalse,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,VIPTrue,VIPFalse
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,False,1,0,1,...,0,1,0,1,0,0,0,1,1,0
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,True,1,0,0,...,1,0,0,1,0,0,0,1,1,0
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,...,0,1,0,1,0,0,0,1,0,1
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,...,0,1,0,1,0,0,0,1,1,0
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,True,1,0,0,...,1,0,0,1,0,0,0,1,1,0


## 3. Missing Values (Float and Int) Imputation

There are a few columns which has null values in either float or int format, some machine learning models can't accept any null data

In [16]:
# age will be filled with average value
spaceship_titanic['Age'] = spaceship_titanic['Age'].fillna(spaceship_titanic['Age'].mean())

In [17]:
spaceship_titanic['RoomService'] = spaceship_titanic['RoomService'].fillna(0)
spaceship_titanic['FoodCourt'] = spaceship_titanic['FoodCourt'].fillna(0)
spaceship_titanic['ShoppingMall'] = spaceship_titanic['ShoppingMall'].fillna(0)
spaceship_titanic['Spa'] = spaceship_titanic['Spa'].fillna(0)
spaceship_titanic['VRDeck'] = spaceship_titanic['VRDeck'].fillna(0)

In [18]:
spaceship_titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12970 entries, 0001_01 to 9277_01
Columns: 1922 entries, Age to VIPFalse
dtypes: float64(6), int64(1), object(1), uint8(1914)
memory usage: 25.1+ MB


## 4. Map Target Values to Binary

In [19]:
spaceship_titanic['Transported'] = spaceship_titanic['Transported'].map({True: 1, False: 0})

## 5. Separate Train and Test Data

In [20]:
cleaned_train = spaceship_titanic[~spaceship_titanic['Transported'].isna()]
cleaned_test = spaceship_titanic[spaceship_titanic['Transported'].isna()]

In [21]:
cleaned_train

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,IsAlone,DeckA,DeckB,...,Earth,Europa,Mars,CryoSleepTrue,CryoSleepFalse,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,VIPTrue,VIPFalse
PassengerId,,,,,,,,,,,,,,,,,,,,,
0001_01,39.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,1,...,0,1,0,1,0,0,0,1,1,0
0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1.0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,0.0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,0.0,0,1,0,...,0,1,0,1,0,0,0,1,1,0
0004_01,16.0,303.0,70.0,151.0,565.0,2.0,1.0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,0.0,1,1,0,...,0,1,0,1,0,1,0,0,0,1
9278_01,18.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0,...,1,0,0,0,1,0,1,0,1,0
9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,1.0,1,0,0,...,1,0,0,1,0,0,0,1,1,0


In [22]:
cleaned_train[cleaned_train.isnull().any(axis=1)]

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,IsAlone,DeckA,DeckB,...,Earth,Europa,Mars,CryoSleepTrue,CryoSleepFalse,55 Cancri e,PSO J318.5-22,TRAPPIST-1e,VIPTrue,VIPFalse
PassengerId,,,,,,,,,,,,,,,,,,,,,


In [23]:
X_train = cleaned_train.drop('Transported', axis=1)
y_train = cleaned_train['Transported']

In [24]:
X_test = cleaned_test.drop('Transported', axis=1)

## 5. Trying Out Different Types of Models

## **SVM**

In [25]:
# from sklearn import svm

In [26]:
# svccl = svm.SVC()
# svccl.fit(X_train, y_train)
# y_pred = svccl.predict(X_test)

In [27]:
# submission = data_test[['PassengerId']]
# submission['Transported'] = y_pred
# submission['Transported'] = submission['Transported'].map({1: True, 0: False})
# submission

In [28]:
# submission.to_csv('submission.csv', index=False)

## **XGBoost**

In [29]:
import xgboost as xgb

In [30]:
# commented is my best result so far
# xgbcl = xgb.XGBClassifier(n_estimators=50, learning_rate=0.1)
xgbcl = xgb.XGBClassifier(
    random_state=42,
    n_estimators=50, 
    learning_rate=0.1
)
xgbcl.fit(X_train, y_train)
y_pred = xgbcl.predict(X_test)
print(xgbcl.score(X_train, y_train))

0.8341194064189578


### XGB HyperParameter Tuning

In [31]:
# from skopt import BayesSearchCV
# from skopt.callbacks import DeadlineStopper, DeltaYStopper
# from skopt.space import Real, Categorical, Integer
# from time import time
# import pprint

In [32]:
# search_spaces = {
#     'learning_rate': Real(0.01, 1.0, 'uniform'),
#     'max_depth': Integer(2, 12),
#     'subsample': Real(0.1, 1.0, 'uniform'),
#     'colsample_bytree': Real(0.1, 1.0, 'uniform'), # subsample ratio of columns by tree
#     'reg_lambda': Real(1e-9, 100., 'uniform'), # L2 regularization
#     'reg_alpha': Real(1e-9, 100., 'uniform'), # L1 regularization
#     'n_estimators': Integer(50, 5000)
# }

In [33]:
# opt = BayesSearchCV(
#     estimator=xgbcl,                                    
#     search_spaces=search_spaces,                      
#     scoring="neg_log_loss",                             
#     cv=10, # 10-fold cv                     
#     n_iter=120, # max number of trials
#     n_points=1, # number of hyperparameter sets evaluated at the same time
#     n_jobs=1, # number of jobs
#     iid=False, # if not iid it optimizes on the cv score
#     return_train_score=False,                         
#     refit=False,                                      
#     optimizer_kwargs={'base_estimator': 'GP'}, # optmizer parameters: we use Gaussian Process (GP)
#     random_state=0 # random state for replicability
# )                                   

In [34]:
# # Reporting util for different optimizers
# def report_perf(optimizer, X, y, title="model", callbacks=None):
#     """
#     A wrapper for measuring time and performances of different optmizers
    
#     optimizer = a sklearn or a skopt optimizer
#     X = the training set 
#     y = our target
#     title = a string label for the experiment
#     """
#     start = time()
    
#     if callbacks is not None:
#         optimizer.fit(X, y, callback=callbacks)
#     else:
#         optimizer.fit(X, y)
        
#     d = pd.DataFrame(optimizer.cv_results_)
#     best_score = optimizer.best_score_
#     best_score_std = d.iloc[optimizer.best_index_].std_test_score
#     best_params = optimizer.best_params_
    
#     print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
#            + u"\u00B1"+" %.3f") % (time() - start, 
#                                    len(optimizer.cv_results_['params']),
#                                    best_score,
#                                    best_score_std))    
#     print('Best parameters:')
#     pprint.pprint(best_params)
#     print()
#     return best_params

In [35]:
# # Running the optimizer
# overdone_control = DeltaYStopper(delta=0.0001)
# time_limit_control = DeadlineStopper(total_time=60*60*4)

# best_params = report_perf(
#     opt, X_train, y_train,'XGBoost_classification', 
#     callbacks=[overdone_control, time_limit_control]
# )

# # OrderedDict([('colsample_bytree', 0.9704181023958698),
# #              ('learning_rate', 0.7589873769816291),
# #              ('max_depth', 6),
# #              ('n_estimators', 962),
# #              ('reg_alpha', 23.854638478198865),
# #              ('reg_lambda', 85.65442013958634),
# #              ('subsample', 0.5030578041377357)])

In [36]:
## this turns to be less accurate than the default one, lol
## 0.79541

# xgbcl_best = xgb.XGBClassifier(
#     colsample_bytree=0.9704181023958698,
#     learning_rate=0.758987376981629,
#     max_depth=6,
#     n_estimators=962,
#     reg_alpha=23.854638478198865,
#     reg_lambda=85.65442013958634,
#     subsample=0.5030578041377357
# )
# xgbcl_best.fit(X_train, y_train)
# print(xgbcl_best.score(X_train, y_train))
# y_pred = xgbcl_best.predict(X_test)

## 6. Create Submission DataFrame

In [37]:
submission = data_test[['PassengerId']]
submission['Transported'] = y_pred
submission['Transported'] = submission['Transported'].map({1: True, 0: False})
submission

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,False


In [38]:
submission.to_csv('submission.csv', index=False)